In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("data/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df.SPECIAL_CONSIDERATIONS.value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [3]:
application_df.STATUS.value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME","STATUS"])

application_df.head(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,0,N,5000,1
6,T3,Independent,C1000,Preservation,Trust,100000-499999,N,31452,1
7,T3,Independent,C2000,Preservation,Trust,10M-50M,N,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1-9999,N,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0


In [5]:
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
count = application_df.APPLICATION_TYPE.value_counts()

count

# Determine which values to replace if counts are less than ...?
replace_application = list(count[count < 300].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classif_count = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classif_count[classif_count < 1880].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists
print(application_df.dtypes)
application_cat = application_df.dtypes[application_df.dtypes =="object"].index.tolist()


print()
print(application_cat)

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()



,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)



In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Attempt 1

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  22
hidden_nodes_layer2 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 22)                946       
_________________________________________________________________
dense_4 (Dense)              (None, 18)                414       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 19        
Total params: 1,379
Trainable params: 1,379
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#define the checkpoints and filepath
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/Optimized_weights.{epoch:02d}.hdf5"


# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


#Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)


In [20]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=20,callbacks=[cp_callback])

Epoch 1/20
804/804 [==============================] - 1s 856us/step - loss: 0.5942 - accuracy: 0.7068
Epoch 2/20
804/804 [==============================] - 1s 846us/step - loss: 0.5672 - accuracy: 0.7244
Epoch 3/20
804/804 [==============================] - 1s 852us/step - loss: 0.5613 - accuracy: 0.7269
Epoch 4/20
804/804 [==============================] - 1s 858us/step - loss: 0.5575 - accuracy: 0.7273
Epoch 5/20
764/804 [===========================>..] - ETA: 0s - loss: 0.5551 - accuracy: 0.7289
Epoch 00005: saving model to checkpoints/Optimized_weights.05.hdf5
804/804 [==============================] - 1s 868us/step - loss: 0.5554 - accuracy: 0.7291
Epoch 6/20
804/804 [==============================] - 1s 972us/step - loss: 0.5535 - accuracy: 0.7306
Epoch 7/20
804/804 [==============================] - 1s 920us/step - loss: 0.5528 - accuracy: 0.7302
Epoch 8/20
804/804 [==============================] - 1s 910us/step - loss: 0.5516 - accuracy: 0.7295
Epoch 9/20
804/804 [============

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5512 - accuracy: 0.7285
Loss: 0.5512223839759827, Accuracy: 0.7285131216049194


# Attempt 2


In [61]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  22
hidden_nodes_layer2 = 18
hidden_nodes_layer3= 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#3rd layer added
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="softmax"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 22)                946       
_________________________________________________________________
dense_43 (Dense)             (None, 18)                414       
_________________________________________________________________
dense_44 (Dense)             (None, 6)                 114       
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 7         
Total params: 1,481
Trainable params: 1,481
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#define the checkpoints and filepath
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/Optimized_weights.{epoch:02d}.hdf5"


# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


#Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)


In [63]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=20,callbacks=[cp_callback])

Epoch 1/20
804/804 [==============================] - 1s 918us/step - loss: 0.6058 - accuracy: 0.6942
Epoch 2/20
804/804 [==============================] - 1s 897us/step - loss: 0.5766 - accuracy: 0.7281
Epoch 3/20
804/804 [==============================] - 1s 966us/step - loss: 0.5748 - accuracy: 0.7290
Epoch 4/20
804/804 [==============================] - 1s 932us/step - loss: 0.5725 - accuracy: 0.7301
Epoch 5/20
765/804 [===========================>..] - ETA: 0s - loss: 0.5671 - accuracy: 0.7312
Epoch 00005: saving model to checkpoints/Optimized_weights.05.hdf5
804/804 [==============================] - 1s 941us/step - loss: 0.5669 - accuracy: 0.7307
Epoch 6/20
804/804 [==============================] - 1s 969us/step - loss: 0.5620 - accuracy: 0.7301
Epoch 7/20
804/804 [==============================] - 1s 938us/step - loss: 0.5598 - accuracy: 0.7301
Epoch 8/20
804/804 [==============================] - 1s 946us/step - loss: 0.5579 - accuracy: 0.7316
Epoch 9/20
804/804 [============

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7311
Loss: 0.5547831654548645, Accuracy: 0.7310787439346313


# Attempt 3

In [55]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  30
hidden_nodes_layer2 = 20
hidden_nodes_layer3= 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#3rd layer added
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 30)                1290      
_________________________________________________________________
dense_39 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_40 (Dense)             (None, 12)                252       
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 13        
Total params: 2,175
Trainable params: 2,175
Non-trainable params: 0
_________________________________________________________________


In [56]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#define the checkpoints and filepath
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [57]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


#Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)


In [58]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 896us/step - loss: 0.6004 - accuracy: 0.6965
Epoch 2/50
804/804 [==============================] - 1s 881us/step - loss: 0.5679 - accuracy: 0.7250
Epoch 3/50
804/804 [==============================] - 1s 909us/step - loss: 0.5609 - accuracy: 0.7269
Epoch 4/50
804/804 [==============================] - 1s 915us/step - loss: 0.5567 - accuracy: 0.7287
Epoch 5/50
736/804 [==========================>...] - ETA: 0s - loss: 0.5535 - accuracy: 0.7289
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 924us/step - loss: 0.5545 - accuracy: 0.7286
Epoch 6/50
804/804 [==============================] - 1s 916us/step - loss: 0.5536 - accuracy: 0.7315
Epoch 7/50
804/804 [==============================] - 1s 922us/step - loss: 0.5520 - accuracy: 0.7302
Epoch 8/50
804/804 [==============================] - 1s 903us/step - loss: 0.5511 - accuracy: 0.7312
Epoch 9/50
804/804 [======================

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7294
Loss: 0.552854597568512, Accuracy: 0.7294460535049438


In [65]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")